In [1]:
import os
import cv2

In [33]:
dirname = 'att'

In [35]:
frames = [589, 590, 591, 592]
ids = [19, 24, 16]
colors = [(255, 0, 0), (0, 0, 255), (100,100,100)]
colors_ = [(0, 0, 255), (255, 0, 0), (100,100,100)]
dirlist = os.listdir(dirname)
for i in range(len(dirlist)):
    if 'txt' in dirlist[i]:
        txtname = dirlist.pop(i)
        break
dirlist = sorted(dirlist)
with open(os.path.join(dirname, txtname), 'r') as f:
    lines = f.readlines()
    lines = [line.strip().split(',') for line in lines]
bbox = {frame:{} for frame in frames}
for line in lines:
    if int(line[0]) in frames and int(line[1]) in ids:
        bbox[int(line[0])].update({
            int(line[1]) : [round(float(x)) for x in line[2:6]]
        })
for i, frame in enumerate(frames):
    imgName = f'000{frame}.jpg'
    img = cv2.imread(os.path.join(dirname, imgName))
    for j, id in enumerate(ids):
        img = cv2.rectangle(
            img, 
            (bbox[frame][id][0], bbox[frame][id][1]), 
            (bbox[frame][id][0]+bbox[frame][id][2], bbox[frame][id][1]+bbox[frame][id][3]),
            color=colors_[j],
            thickness=2
        )
        if frame-1 in bbox:
            drawrect(
                img, 
                (bbox[frame-1][id][0], bbox[frame-1][id][1]), 
                (bbox[frame-1][id][0]+bbox[frame-1][id][2], bbox[frame-1][id][1]+bbox[frame-1][id][3]),
                color=colors[j],
                thickness=2,
                style='aa'
            )
    img = img[100:400, 500:750, :]
    cv2.imwrite(os.path.join(dirname, imgName[:-4]+'_box__.jpg'), img)

In [29]:
import cv2
import numpy as np
def drawline(img,pt1,pt2,color,thickness=1,style='dotted',gap=4):
    dist =((pt1[0]-pt2[0])**2+(pt1[1]-pt2[1])**2)**.5
    pts= []
    for i in  np.arange(0,dist,gap):
        r=i/dist
        x=int((pt1[0]*(1-r)+pt2[0]*r)+.5)
        y=int((pt1[1]*(1-r)+pt2[1]*r)+.5)
        p = (x,y)
        pts.append(p)

    if style=='dotted':
        for p in pts:
            cv2.circle(img,p,thickness,color,-1)
    else:
        s=pts[0]
        e=pts[0]
        i=0
        for p in pts:
            s=e
            e=p
            if i%2==1:
                cv2.line(img,s,e,color,thickness)
            i+=1

def drawpoly(img,pts,color,thickness=1,style='dotted',):
    s=pts[0]
    e=pts[0]
    pts.append(pts.pop(0))
    for p in pts:
        s=e
        e=p
        drawline(img,s,e,color,thickness,style)

def drawrect(img,pt1,pt2,color,thickness=1,style='dotted'):
    pts = [pt1,(pt2[0],pt1[1]),pt2,(pt1[0],pt2[1])] 
    drawpoly(img,pts,color,thickness,style)

In [47]:
dirlist = sorted(dirlist)